In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
events = pd.read_csv('/kaggle/input/recsys-df/events.csv')

In [90]:
events

,user_id,item_id,rating,timestamp
0,0,1505,4,0
1,0,3669,3,1
2,0,584,4,2
3,0,3390,3,3
4,0,2885,4,4
...,...,...,...,...
894144,6039,1453,4,223
894145,6039,613,5,224
894146,6039,1548,4,225
894147,6039,241,4,226


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [78]:
def train_test_split_custom(events):
    user_last_event = events \
                    .groupby(by = 'user_id', as_index = False).max().loc[: , ['user_id', 'timestamp']]
    events_with_taget_flg = events.merge(user_last_event, on = ['user_id'], how = 'left')

    events_with_taget_flg['target_flg'] = list(map(lambda x, y: 1 if x == y else 0, 
                                            events_with_taget_flg.timestamp_y, events_with_taget_flg.timestamp_x))
    train = events_with_taget_flg[events_with_taget_flg.target_flg == 0]
    test = events_with_taget_flg[events_with_taget_flg.target_flg == 1]
    return train, test

In [79]:
train, test = train_test_split_custom(events)

In [81]:
train

,user_id,item_id,rating,timestamp_x,timestamp_y,target_flg
0,0,1505,4,0,326,0
1,0,3669,3,1,326,0
2,0,584,4,2,326,0
3,0,3390,3,3,326,0
4,0,2885,4,4,326,0
...,...,...,...,...,...,...
894143,6039,2510,5,222,227,0
894144,6039,1453,4,223,227,0
894145,6039,613,5,224,227,0
894146,6039,1548,4,225,227,0


In [82]:
user_item_matrix = train.pivot_table(index='user_id', columns='item_id', values='rating')

In [83]:
user_item_matrix

item_id,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [84]:
user_item_matrix = user_item_matrix.fillna(0).to_numpy()

In [85]:
user_item_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 3.],
       [0., 0., 0., ..., 0., 0., 2.]])

In [86]:
mean_user_rating = np.mean(user_item_matrix, axis=1).reshape(-1, 1)
normalized_matrix = user_item_matrix - mean_user_rating

In [87]:
# Fit the KNN model
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(normalized_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [88]:
dct={}
for target_user_index in range(len(events['user_id'].unique())):
    distances, indices = knn.kneighbors(normalized_matrix[target_user_index].reshape(1, -1), n_neighbors=150)
    neighbors_ratings = user_item_matrix[indices.flatten()]
    predicted_ratings = neighbors_ratings.mean(axis=0)
    unrated_items = np.where(user_item_matrix[target_user_index] == 0)[0]
    recommended_items = unrated_items[np.argsort(predicted_ratings[unrated_items])[::-1]]
    recommended_items = recommended_items[:50]
#     for item in recommended_items:
#         feature_df.loc[(feature_df['user_id'] == target_user_index)&(feature_df['item_id'] == item)]['knn_score'] = 1
    dct[target_user_index] = recommended_items

In [91]:
item_features = pd.read_csv('/kaggle/input/recsys-df/item_features.csv')


In [97]:
test.loc[test['user_id']==0]['item_id'].values[0]

3360

In [98]:
np.random.seed(28)
# feature_df = pd.DataFrame(data={'user_id': [], 'item_id': []})
users = []
items = []
items_neg = []
users_neg = []
all_items = set(item_features['item_id'].unique())
for user in dct.keys():
    items.extend(dct[user])
    users.extend([user]*50)
    candidates = list(all_items - set(dct[user]))
#     neg_sample = np.random.choice(candidates, 50)
#     items_neg.extend(neg_sample)
    if test.loc[test['user_id']==user]['item_id'].values[0] not in dct[user]:
        neg_sample = np.random.choice(candidates, 49)
        items_neg.extend(neg_sample)
        items_neg.append(test.loc[test['user_id']==user]['item_id'].values[0])
    else:
        neg_sample = np.random.choice(candidates, 50)
        items_neg.extend(neg_sample)
    users_neg.extend([user]*50)
    

In [99]:
pos_sample = pd.DataFrame(data={'user_id': users, 'item_id': items})
pos_sample['knn_pred'] = 1
neg_sample = pd.DataFrame(data={'user_id': users_neg, 'item_id': items_neg})
neg_sample['knn_pred'] = 0

features_df = pd.concat([pos_sample, neg_sample])

In [100]:
features_df

,user_id,item_id,knn_pred
0,0,2719,1
1,0,1800,1
2,0,2243,1
3,0,2848,1
4,0,3045,1
...,...,...,...
301995,6039,739,0
301996,6039,244,0
301997,6039,795,0
301998,6039,54,0


In [101]:
top_50_items = train.groupby('item_id').agg({'user_id': "count"})['user_id'].sort_values(ascending=False).reset_index()[:50]['item_id'].values
features_df['if_popular'] = 0
for item in top_50_items:
    features_df.loc[features_df['item_id'] == item, 'if_popular'] = 1


In [102]:
user_features = pd.read_csv('/kaggle/input/recsys-df/user_features.csv')
features_df = features_df.merge(user_features, on='user_id', how='left')\
.merge(item_features, on='item_id', how='left')

In [103]:
features_df['gender_int'] = 0
features_df.loc[features_df['gender'] == 'M', 'gender_int'] = 1

In [104]:
features_df['if_popular'].mean()

0.024980132450331125

In [105]:
features_df['gender_int'].mean()

0.7170529801324503

In [106]:
features_df = features_df.drop('gender', axis=1)

In [107]:
item_avg_rating = train.groupby('item_id')['rating'].mean().reset_index()
item_avg_rating.columns = ['item_id', 'item_avg_rating']

In [108]:
user_avg_rating = train.groupby('user_id')['rating'].mean().reset_index()
user_avg_rating.columns = ['user_id', 'user_avg_rating']

In [109]:
features_df = features_df.merge(item_avg_rating, on='item_id', how='left')

In [110]:
features_df = features_df.merge(user_avg_rating, on='user_id', how='left')

In [111]:
tmp = train.merge(item_features, on = ['item_id'], how = 'left')
for i in range(18): 
    genre_col = f'genre_{i}'
    user_genre_counts = tmp.groupby('user_id')[genre_col].sum().reset_index()
    user_genre_counts.columns = ['user_id', f'user_genre_{i}_count']
    user_features = user_features.merge(user_genre_counts, on='user_id', how='left')

In [112]:
user_features = user_features.drop(['gender', 'age'], axis=1)

In [113]:
features_df = features_df.merge(user_features, on='user_id', how='left')

In [114]:
features_df

,user_id,item_id,knn_pred,if_popular,age,genre_0,genre_1,genre_2,genre_3,genre_4,...,user_genre_8_count,user_genre_9_count,user_genre_10_count,user_genre_11_count,user_genre_12_count,user_genre_13_count,user_genre_14_count,user_genre_15_count,user_genre_16_count,user_genre_17_count
0,0,2719,1,0,35,0,0,0,0,0,...,19,4,22,18,13,54,47,65,17,4
1,0,1800,1,0,35,0,0,0,0,0,...,19,4,22,18,13,54,47,65,17,4
2,0,2243,1,0,35,0,0,0,0,0,...,19,4,22,18,13,54,47,65,17,4
3,0,2848,1,0,35,0,0,0,0,0,...,19,4,22,18,13,54,47,65,17,4
4,0,3045,1,0,35,0,0,0,0,0,...,19,4,22,18,13,54,47,65,17,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603995,6039,739,0,0,35,0,0,0,0,0,...,7,9,17,2,10,38,72,38,8,3
603996,6039,244,0,0,35,0,0,0,0,0,...,7,9,17,2,10,38,72,38,8,3
603997,6039,795,0,0,35,0,0,0,0,1,...,7,9,17,2,10,38,72,38,8,3
603998,6039,54,0,0,35,0,0,0,0,1,...,7,9,17,2,10,38,72,38,8,3


In [115]:
features_df['target'] = 0
for i in range(test.shape[0]):
    user=test['user_id'].values[i]
    item = test['item_id'].values[i]
    features_df.loc[(features_df['user_id'] == user)&(features_df['item_id'] == item), 'target'] =1

In [116]:
features_df['target'].mean()

0.010134105960264901

In [117]:
import catboost

In [119]:
from catboost import CatBoostClassifier, Pool

In [120]:
X = features_df.drop(['user_id', 'item_id', 'target'], axis=1)
y = features_df['target']
ctb = CatBoostClassifier()
ctb.fit(Pool(X, y))

Learning rate set to 0.158649
0:	learn: 0.3986172	total: 156ms	remaining: 2m 35s
1:	learn: 0.2447042	total: 244ms	remaining: 2m 1s
2:	learn: 0.1413888	total: 341ms	remaining: 1m 53s
3:	learn: 0.0971659	total: 438ms	remaining: 1m 49s
4:	learn: 0.0762499	total: 527ms	remaining: 1m 44s
5:	learn: 0.0652950	total: 618ms	remaining: 1m 42s
6:	learn: 0.0590345	total: 715ms	remaining: 1m 41s
7:	learn: 0.0553757	total: 805ms	remaining: 1m 39s
8:	learn: 0.0527216	total: 908ms	remaining: 1m 39s
9:	learn: 0.0510911	total: 1.01s	remaining: 1m 40s
10:	learn: 0.0500681	total: 1.14s	remaining: 1m 42s
11:	learn: 0.0492879	total: 1.3s	remaining: 1m 47s
12:	learn: 0.0487529	total: 1.46s	remaining: 1m 51s
13:	learn: 0.0484034	total: 1.6s	remaining: 1m 53s
14:	learn: 0.0481343	total: 1.76s	remaining: 1m 55s
15:	learn: 0.0479272	total: 1.91s	remaining: 1m 57s
16:	learn: 0.0476922	total: 2.06s	remaining: 1m 59s
17:	learn: 0.0475377	total: 2.21s	remaining: 2m
18:	learn: 0.0474509	total: 2.35s	remaining: 2m 1s


Inference

In [130]:
user_item_matrix = events.pivot_table(index='user_id', columns='item_id', values='rating')

In [131]:
user_item_matrix = user_item_matrix.fillna(0).to_numpy()

In [132]:
mean_user_rating = np.mean(user_item_matrix, axis=1).reshape(-1, 1)
normalized_matrix = user_item_matrix - mean_user_rating

In [133]:
# Fit the KNN model
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(normalized_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [134]:
dct={}
for target_user_index in range(len(events['user_id'].unique())):
    distances, indices = knn.kneighbors(normalized_matrix[target_user_index].reshape(1, -1), n_neighbors=150)
    neighbors_ratings = user_item_matrix[indices.flatten()]
    predicted_ratings = neighbors_ratings.mean(axis=0)
    unrated_items = np.where(user_item_matrix[target_user_index] == 0)[0]
    recommended_items = unrated_items[np.argsort(predicted_ratings[unrated_items])[::-1]]
    recommended_items = recommended_items[:50]
#     for item in recommended_items:
#         feature_df.loc[(feature_df['user_id'] == target_user_index)&(feature_df['item_id'] == item)]['knn_score'] = 1
    dct[target_user_index] = recommended_items

In [135]:
np.random.seed(28)
# feature_df = pd.DataFrame(data={'user_id': [], 'item_id': []})
users = []
items = []
items_neg = []
users_neg = []
all_items = set(item_features['item_id'].unique())
for user in dct.keys():
    items.extend(dct[user])
    users.extend([user]*50)
    candidates = list(all_items - set(dct[user]))
#     neg_sample = np.random.choice(candidates, 50)
#     items_neg.extend(neg_sample)
    if test.loc[test['user_id']==user]['item_id'].values[0] not in dct[user]:
        neg_sample = np.random.choice(candidates, 49)
        items_neg.extend(neg_sample)
        items_neg.append(test.loc[test['user_id']==user]['item_id'].values[0])
    else:
        neg_sample = np.random.choice(candidates, 50)
        items_neg.extend(neg_sample)
    users_neg.extend([user]*50)

In [143]:
pos_sample = pd.DataFrame(data={'user_id': users, 'item_id': items})
pos_sample['knn_pred'] = 1
neg_sample = pd.DataFrame(data={'user_id': users_neg, 'item_id': items_neg})
neg_sample['knn_pred'] = 0

features_df = pd.concat([pos_sample, neg_sample])

In [144]:
top_50_items = events.groupby('item_id').agg({'user_id': "count"})['user_id'].sort_values(ascending=False).reset_index()[:50]['item_id'].values
features_df['if_popular'] = 0
for item in top_50_items:
    features_df.loc[features_df['item_id'] == item, 'if_popular'] = 1
user_features = pd.read_csv('/kaggle/input/recsys-df/user_features.csv')
   
features_df = features_df.merge(user_features, on='user_id', how='left')\
.merge(item_features, on='item_id', how='left')

features_df['gender_int'] = 0
features_df.loc[features_df['gender'] == 'M', 'gender_int'] = 1

In [145]:
features_df = features_df.drop('gender', axis=1)

item_avg_rating = events.groupby('item_id')['rating'].mean().reset_index()
item_avg_rating.columns = ['item_id', 'item_avg_rating']

user_avg_rating = events.groupby('user_id')['rating'].mean().reset_index()
user_avg_rating.columns = ['user_id', 'user_avg_rating']

features_df = features_df.merge(item_avg_rating, on='item_id', how='left')
features_df = features_df.merge(user_avg_rating, on='user_id', how='left')

In [146]:
tmp = events.merge(item_features, on = ['item_id'], how = 'left')
for i in range(18): 
    genre_col = f'genre_{i}'
    user_genre_counts = tmp.groupby('user_id')[genre_col].sum().reset_index()
    user_genre_counts.columns = ['user_id', f'user_genre_{i}_count']
    user_features = user_features.merge(user_genre_counts, on='user_id', how='left')
    
user_features = user_features.drop(['gender', 'age'], axis=1)
features_df = features_df.merge(user_features, on='user_id', how='left')

In [152]:
X_test = features_df.drop(['user_id', 'item_id'], axis=1)
features_df['preds'] = ctb.predict_proba(X_test)[:,1]

In [161]:
features_df

,user_id,item_id,knn_pred,if_popular,age,genre_0,genre_1,genre_2,genre_3,genre_4,...,user_genre_9_count,user_genre_10_count,user_genre_11_count,user_genre_12_count,user_genre_13_count,user_genre_14_count,user_genre_15_count,user_genre_16_count,user_genre_17_count,preds
0,0,2720,1,0,35,0,0,0,0,0,...,4,22,18,13,54,47,65,17,4,0.000265
1,0,1801,1,0,35,0,0,0,0,0,...,4,22,18,13,54,47,65,17,4,0.000688
2,0,2244,1,0,35,0,0,0,0,0,...,4,22,18,13,54,47,65,17,4,0.000001
3,0,2849,1,0,35,0,0,0,0,0,...,4,22,18,13,54,47,65,17,4,0.000153
4,0,3046,1,0,35,1,1,0,0,0,...,4,22,18,13,54,47,65,17,4,0.001609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603995,6039,1046,0,0,35,1,0,0,0,0,...,9,17,2,10,38,72,38,8,4,0.017871
603996,6039,974,0,0,35,0,0,0,0,0,...,9,17,2,10,38,72,38,8,4,0.014786
603997,6039,2487,0,0,35,0,0,0,0,0,...,9,17,2,10,38,72,38,8,4,0.010552
603998,6039,1634,0,0,35,0,0,0,0,0,...,9,17,2,10,38,72,38,8,4,0.006446


In [169]:
df = features_df.loc[features_df['user_id'] == 2][['preds', 'item_id']]
recs = list(df.sort_values(by='preds', ascending=False)[:10]['item_id'].values)

In [170]:
recs

[650, 2625, 43, 3027, 1937, 2128, 2840, 2693, 876, 493]

In [179]:
a = []
u = []
for user in features_df['user_id'].unique():
    df = features_df.loc[features_df['user_id'] == user][['preds', 'item_id']]
    recs = list(df.sort_values(by='preds', ascending=False)[:10]['item_id'].values)
    recs = ' '.join(str(x) for x in recs)
    a.append(recs)
    u.append(str(user))
fin_pred = pd.DataFrame(data={'user_id':u, 'item_id':a})
fin_pred.to_csv('submission_knn_and_ctb.csv')

In [180]:
fin_pred

,user_id,item_id
0,0,1956 2534 1295 458 1316 3402 3377 45 943 2557
1,1,584 801 1300 1001 1686 1950 839 2938 3637 258
2,2,650 2625 43 3027 1937 2128 2840 2693 876 493
3,3,700 943 3191 2804 1302 3575 2189 2069 212 277
4,4,1868 2980 3207 1191 394 2617 1879 1012 1761 2542
...,...,...
6035,6035,2402 2523 3245 214 2896 28 2159 3216 1792 2953
6036,6036,1583 3431 829 3600 1788 2345 3142 2983 3547 1975
6037,6037,2872 3105 2682 2294 1681 2670 1456 1040 290 2681
6038,6038,1945 2047 1969 2106 163 516 33 1827 2508 970
